<a href="https://colab.research.google.com/github/sigvehaug/DSF-DCBP/blob/main/12_Fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 12 Fitting

In science we develop and create models which sometimes also become theories. Models normally contain adjustable parameteres. Such parameters can be determined from data. This task is called parameter fitting.

In chemical models there are parameters which have been obtained, i.e. fitted from data. Several Python modules, e.g. the stats, the optimize or the symfit modules, have fitting methods. In this notebook we look at some of them.

When fitting data, we do optimisation, i.e. we find some minimum of a function depending on our parameters. This normally include derivatives with respect to the parameters and some optimisation function. Much used optimisation functions are
- Maximum Likelihood
- Least Squares

In Machine Learning or Artificial Intelligence we do the same. We fit parameters, up to trillions, of the model to data. In this field, the function to be optimized is called Loss or Cost function. The loss function is often a least squares.

When you fit parameters, you are often confronted with (at least) three issues.
- Goodness of Fit
- Overfitting
- Local or global minimum



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Fitting data with stats PDFs

The scipy.stats module has many probability density functions which can be used to describe and model data. Let us fit some of the MCR data to gaussian distributions, i.e. normal models.

In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Courses/DSF-DCBP/Data-MCR/P3HT_Abs_data_Teaching.txt' #Data-CCD/4ms_10 av_30 s_Absorbance_10-32-04-868.txt'

````
# Read the MCR data into a dataframe
import pandas as pd
df = pd.read_csv(path, delimiter='\t',header=0)
df.info()````


In [ ]:
import pandas as pd
df = pd.read_csv(path, delimiter='\t', header=0)
df.info()

```df.head()```

In [ ]:
df.head()

```
# Histogram of selected columns
df.hist(df.columns[5], bins=20)
```


In [ ]:
df.hist(df.columns[5], bins=30)

In [ ]:
df.iloc[:,5]

In [ ]:
from scipy.stats import norm
norm.fit(df.iloc[:,5].dropna())

The predefined pdfs in scipy.stats you can see https://docs.scipy.org/doc/scipy/reference/stats.html. Sometimes you want to define your own model, or it is not already implemented, or you don't find the implementation. Then you can write your own model and use the scipy.optimize modul.

## Fitting your own model with scipy.optimize

Let's practice with scipy.optimize. First we generate syntetic data from an exponential funciton with three parameters. Then we add gaussian noise to the data so that it becomes more realistic. Lastly we fit the parameters of a self-defined exponential function to that noisy data.  

```
from scipy.optimize import curve_fit

def func(x, a, b, c):
     return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50) #
y = func(xdata, 2.5, 1.3, 0.5)
plt.plot(xdata, y, 'g-', label='Generated data')
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='Generated data with noise')
plt.legend()
plt.show()
```

In [ ]:
from scipy.optimize import curve_fit
def func(x, a, b, c):
  return a * np.exp(-b*x) + c

xdata = np.linspace(0,4,50)
y = func(xdata, 2.5, 1.3, 0.5)
plt.plot(xdata, y, 'g-', label='Generated data')

np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise

plt.plot(xdata, ydata, 'b-', label='Generated data with noise')
plt.legend()
plt.show()

```
popt, pcov = curve_fit(func, xdata, ydata)
print(popt)
perr = np.sqrt(np.diag(pcov)) # Standard deviation = square root of the variance being on the diagonal of the covariance matrix
plt.plot(xdata, func(xdata, *popt), 'r-',label= \
         'fit: a=%5.3f +- %5.3f, \n b=%5.3f +- %5.3f, \n c=%5.3f +-%5.3f' % \
         (popt[0],perr[0],popt[1],perr[1],popt[2],perr[2]))
plt.xlabel('x')
plt.ylabel('y')
plt.plot(xdata, ydata, 'b+', label='Data')
plt.legend()
plt.show()
perr = np.sqrt(np.diag(pcov)) # Standard deviation = square root of the variance being on the diagonal of the covariance matrix
perr
```

In [ ]:
popt,pcov = curve_fit(func, xdata, ydata)
print(popt)
print(pcov)
perr = np.sqrt(np.diag(pcov))
print(perr)

In [ ]:
plt.plot(xdata, func(xdata, *popt), 'r-',label= \
         'fit: a=%5.3f +- %5.3f, \n b=%5.3f +- %5.3f, \n c=%5.3f +-%5.3f' % \
         (popt[0],perr[0],popt[1],perr[1],popt[2],perr[2]))
plt.xlabel('x')
plt.ylabel('y')
plt.plot(xdata, ydata, 'b+', label='Data')
plt.legend()
plt.show()
perr = np.sqrt(np.diag(pcov)) # Standard deviation = square root of the variance being on the diagonal of the covariance matrix
perr

So with scipy.optimize you can define your own function/model and fit its parameters to your data. There exist other libraries which makes fitting even more convenient. In the next section we look at one.

## Fitting with symfit

The more comprehensive module for fitting with Python is symfit (https://symfit.readthedocs.io/). By reading it's documentation, you can also learn very compactly the most important things about fitting. Let us do a short tutorial here.


In [ ]:
!pip install symfit

In [ ]:
from symfit import Parameter, Variable, parameters

a = Parameter('a')
b = Parameter('b')
x = Variable('x')
model = a * x + b

In [ ]:
from symfit import Fit
import numpy as np

xdata = np.linspace(0, 100, 100) # From 0 to 100 in 100 steps
a_vec = np.random.normal(15.0, scale=2.0, size=(100,))
b_vec = np.random.normal(100.0, scale=2.0, size=(100,))
ydata = a_vec * xdata + b_vec

fit = Fit(model, xdata, ydata)
fit_result = fit.execute()
fit_result.minimizer_output

In [ ]:
fit_result.value(a)

# Voluntary Exercise

If you have time and interest, study and play with the notebook provided by the [FemtoMat](https://banerji.dcbp.unibe.ch/) group.
- https://github.com/sigvehaug/DSF-DCBP/blob/60f69ef96a4c394f6df5030259f6ea99ddbdc4b2/Fitting_Example.ipynb